# Игра с клетками

В этой задаче поиграем в игру на поле $m\times n$, клетки которого могут быть в двух состояниях: «горящие» или «потухшие». При нажатии клетка и её четыре соседа по диагонали меняют состояние. Цель игры — потушить все клетки.

![Игра с квадратами](squares-example.png)

Игра обманчиво проста, но победить в ней не так-то легко. Попробуйте [поиграть.](https://htmlpreview.github.io/?https://github.com/dkosolobov/squaregame/blob/main/squaregame.html)

В этой задаче вам надо разработать программу, которая с помощью систем уравнений в поле $Z_2$ по заданному игровому полю определяет, можно ли победить и как именно.

**1. Система $4 \times 4$.** Допишите функцию, которая получает на вход $4\times 4$ матрицу $A$ над полем $\mathbb{Z}_2$ и столбец $b$ размера 4 (т.е. $A \in \mathbb{Z}_2^{4\times 4}$ и $b \in \mathbb{Z}_2^4$) и возврашает `True`, если система $Ax=b$ имеет решение в поле $\mathbb{Z}_2$, и `False` в противном случае.  Можно пропустить это задание, если вы выполните следующее. Техническое напоминание: код обмена строк матрицы выглядит как `A[[i,j]]=A[[j,i]]`.

In [1]:
import numpy as np

def is_4x4_system_solvable(A, b):
    """
    Проверяет существование решения системы Ax = b в поле {0,1},
    """
    if A.shape != (4,4) or b.shape != (4,):
        raise ValueError("Матрица A должна быть 4x4, а b — размером 4")

    A = A.copy() % 2
    b = b.copy() % 2
    n = 4

    leader_row = 0
    for column in range(n):
        leader = None
        for i in range(leader_row, n):
            if A[i, column] == 1:
                leader = i
                break

        if leader is None:
            continue

        if leader != leader_row:
            A[[leader_row, leader]] = A[[leader, leader_row]]
            b[[leader_row, leader]] = b[[leader, leader_row]]

        for i in range(leader_row + 1, n):
            if A[i, column] == 1:
                A[i] = (A[i] + A[leader_row]) % 2
                b[i] = (b[i] + b[leader_row]) % 2

        leader_row += 1

    leaders = []
    for i in range(n):
        leader_column = None
        for c in range(n):
            if A[i, c] == 1:
                leader_column = c
                break

        if leader_column is not None:
            leaders.append((i, leader_column))

    for leader_i, leader_c in reversed(leaders):
        for i in range(i):
            if A[i, leader_c] == 1:
                A[i] = (A[i] + A[leader_i]) % 2
                b[i] = (b[i] + b[leader_i]) % 2

    for i in range(n):
        if np.all(A[i] == 0) and b[i] == 1:
            return False

    return True

**2. Система $n \times n$.** Допишите функцию, которая получает на вход $n\times n$ матрицу $A$ над полем $\mathbb{Z}_2$ и столбец $b$ размера $n$ (т.е. $A \in \mathbb{Z}_2^{n\times n}$ и $b \in \mathbb{Z}_2^n$) и возврашает `True`, если система $Ax=b$ имеет решение в поле $\mathbb{Z}_2$, и `False` в противном случае.

In [2]:
import numpy as np

def is_nxn_system_solvable(A, b):
    """
    Проверяет существование решения системы Ax = b в поле {0,1}.
    """
    A = A.copy() % 2
    b = b.copy() % 2
    rows, columns = A.shape

    leader_row = 0
    for column in range(columns):
        leader = None

        for i in range(leader_row, rows):
            if A[i, column] == 1:
                leader = i
                break

        if leader is None:
            continue

        if leader != leader_row:
            A[[leader_row, leader]] = A[[leader, leader_row]]
            b[[leader_row, leader]] = b[[leader, leader_row]]

        for i in range(leader_row + 1, rows):
            if A[i, column] == 1:
                A[i] = (A[i] + A[leader_row]) % 2
                b[i] = (b[i] + b[leader_row]) % 2

        leader_row += 1

    for i in range(rows):
        if np.all(A[i] == 0) and b[i] == 1:
            return False

    return True

**3. Разрешимость игры.** Допишите функцию, которая возвращает `True`, если в заданной игре можно победить, и `False` в противном случае. Игровое поле представлено матрицей нулей и единиц размера $m \times n$, в которой ноль означает потухшую клетку, единица — горящую. Подсказка: с помощью `is_4x4_system_solvable` можно разрешать игровые поля размеров $2\times 2$, $4 \times 1$ и $1 \times 4$.

In [3]:
import numpy as np

def is_game_solvable(GameField):
    '''Проверяет разрешимость игрового поля GameField[0:m,0:n] из нулей и единиц'''
    GameField = GameField.copy() % 2  # Приводим к полю Z2
    [m, n] = GameField.shape

    A = build_matrix_A(m, n)
    b = GameField.flatten()

    return is_nxn_system_solvable(A, b)

def build_matrix_A(m, n):
    '''создать матрицу A для игрового поля размером m x n'''
    size = m * n
    A = np.zeros((size, size), dtype=int)

    for i in range(m):
        for j in range(n):
            index = i * n + j
            A[index, index] = 1

            #the upper left neighbor
            if i > 0 and j > 0:
                A[index, (i - 1) * n + (j - 1)] = 1

            #the upper right neighbor
            if i > 0 and j < n - 1:
                A[index, (i - 1) * n + (j + 1)] = 1

            #the lower left neighbor
            if i < m - 1 and j > 0:
                A[index, (i + 1) * n + (j - 1)] = 1

            #the lower right neighbor
            if i < m - 1 and j < n - 1:
                A[index, (i + 1) * n + (j + 1)] = 1

    return A

**4. Решение игры.** Если в игре можно победить, то хочется знать как. Допишите функцию, возвращающую список клеток, которые надо нажать для победы, и возвращающую `None`, если победить нельзя. Клетки задаются парами чисел `[x,y]`; например, список `[[0,0], [0,2], [1,1]]`  означает, что надо нажать на левую верхнюю клетку, затем третью клетку в верхнем ряду, а затем вторую клетку во втором ряду.

In [4]:
def solve_game(GameField):
    '''Найти решение игрового поля GameField[0:m,0:n] из нулей и единиц'''
    GameField = GameField.copy()
    [m, n] = GameField.shape

    if not is_game_solvable(GameField):
        return None

    A = np.zeros((m * n, m * n), dtype=int)
    b = GameField.flatten()

    for i in range(m):
        for j in range(n):
            index = i * n + j
            A[index, index] = 1
            diagonal = [(-1, -1), (-1, 1), (1, -1), (1, 1)]

            for dx, dy in diagonal:
                ni, nj = i + dx, j + dy

                if 0 <= ni < m and 0 <= nj < n:
                    A[index, ni * n + nj] = 1

    solution = gauss(np.column_stack((A, b)), True)

    result = []
    for index, value in enumerate(solution):
        if value == 1:
            result.append([index // n, index % n])

    return result

def gauss(extended_matrix, return_solution):
    matrix = extended_matrix.copy()
    rows, cols = matrix.shape

    for leader_row in range(rows):
        if matrix[leader_row, leader_row] == 0:
            for swap_row in range(leader_row + 1, rows):
                if matrix[swap_row, leader_row] == 1:
                    matrix[[leader_row, swap_row]] = matrix[[swap_row, leader_row]]
                    break

        for row in range(leader_row + 1, rows):
            if matrix[row, leader_row] == 1:
                matrix[row] = (matrix[row] + matrix[leader_row]) % 2

    for leader_row in range(rows - 1, -1, -1):
        for row in range(leader_row - 1, -1, -1):
            if matrix[row, leader_row] == 1:
                matrix[row] = (matrix[row] + matrix[leader_row]) % 2

    if return_solution:
        return matrix[:, -1]

    else:
        return matrix

## Тестирование

Осталось проверить, проходят ли ваши решения первоначальное тестирование. Выполните весь код (Cell -> Run All или Runtime -> Run All или другим способом, работающим в вашей среде) и посмотрите вердикт внизу страницы. В тестирующем коде разбираться не нужно!


In [5]:
import numpy as np
from itertools import chain, product

solvable_4x4 = ({'A': [[0,0,0,1],[0,1,0,1],[0,1,1,0],[1,0,1,0]], 'b':[0,1,0,1]},
    {'A':[[0,1,0,1],[1,0,0,1],[1,1,1,1],[0,0,1,1]],'b': [1,1,0,0]})
unsolvable_4x4 = ({'A':[[0,1,0,1],[1,0,0,1],[1,1,1,1],[0,0,1,1]],'b':[1,1,0,1]},
    {'A':[[1,0,1,0],[0,1,1,0],[1,1,0,0],[0,0,0,0]],'b':[1,1,1,1]},
    {'A':[[1,0,1,1],[0,0,1,0],[0,0,0,0],[1,0,0,1]],'b': [0,1,0,0]})
solvable_nxn = solvable_4x4 + ({'A':[[1]],'b':[1]},{'A':[[1]],'b':[0]},
    {'A':[[0]],'b':[0]},{'A':[[0,0,0],[0,1,1],[1,1,1]],'b':[0,1,1]},
    {'A':[[0,0,0,0,0],[0,0,0,0,1],[0,0,1,1,0],[0,0,0,1,0],[0,0,1,0,1]],'b':[0,0,1,1,0]},
    {'A':[[1,1,0,0,1],[0,1,0,0,0],[0,1,0,1,0],[0,0,0,1,0],[1,1,0,1,1]],'b':[0,0,1,1,1]})
unsolvable_nxn = unsolvable_4x4 + ({'A':[[0]], 'b':[1]},
    {'A':[[0,0,0],[0,1,1],[1,1,1]],'b':[1,1,1]},
    {'A':[[0,0,0,0,0],[0,0,0,0,1],[0,0,1,1,0],[0,0,0,1,0],[0,0,1,0,1]],'b':[0,0,1,1,1]})
solvable_games = ([[0]],[[1]],[[0,0]],[[1,1]],[[1],[1]], [[1,0],[0,1]], [[0],[1],[0],[1]],
    [[0,1],[0,1],[1,0]],[[1,1,1,1,0],[1,1,0,1,1],[1,0,1,0,0]], [[1, 1, 1], [0, 1, 1], [1, 1, 1]],
    [[0,1,0,1],[0,1,0,1],[1,1,1,1],[1,1,1,1],[0,1,0,1]],
    [[0,0,0,0,1,1], [1,1,1,1,1,1], [1,1,1,1,0,0], [0,0,1,1,1,1], [1,1,1,1,1,1], [1,1,0,0,0,0]])
unsolvable_games = ([[0,1],[0,1]], [[1,1],[0,0]], [[1,0],[1,0]], [[0,1],[1,1]], [[0,1,1,1,1],[1,1,1,1,1]],
    [[0,1,0,1],[0,1,0,1],[0,1,0,1],[0,1,0,1]],
    [[0,0,1,1,0],[0,1,1,1,1],[1,1,1,1,1],[1,0,0,1,1],[0,0,1,1,1]],
    [[0,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1]] )

def seq_solves_game(seq, game):
    game = game.copy()
    if seq is None:
        return False
    for i, j in seq:
        for si, sj in [(0,0), (1,1), (1,-1), (-1,1), (-1,-1)]:
            if 0 <= i + si < len(game) and 0 <= j + sj < len(game[0]):
                game[i + si, j + sj] ^= 1
    return all(all(cell == 0 for cell in row) for row in game)

def test_all():
    test_seq = chain((t for t in solvable_4x4 if not is_4x4_system_solvable(
                      np.array(t['A']), np.array(t['b']))),
                   (t for t in unsolvable_4x4 if is_4x4_system_solvable(
                      np.array(t['A']), np.array(t['b']))))
    test = next(test_seq, None)
    if test is not None:
        print('Первое задание не прошло тест:')
        print('A =\n', np.array(test['A']), '\nb =', np.array(test['b']))
    else:
        print('Первое задание прошло все тесты')

    test_seq = chain((t for t in solvable_nxn if not is_nxn_system_solvable(
                      np.array(t['A']), np.array(t['b']))),
                   (t for t in unsolvable_nxn if is_nxn_system_solvable(
                      np.array(t['A']), np.array(t['b']))))
    test = next(test_seq, None)
    if test is not None:
        print('Второе задание не прошло тест:')
        print('A =\n', np.array(test['A']), '\nb =', np.array(test['b']))
    else:
        print('Второе задание прошло все тесты')

    test = next(chain((t for t in solvable_games
                     if not is_game_solvable(np.array(t))),
                    (t for t in unsolvable_games
                     if is_game_solvable(np.array(t)))), None)
    if test is not None:
        print('Третье задание не прошло тест:')
        print(np.array(test))
    else:
        print('Третье задание прошло все тесты')

    test = next(chain((t for t in solvable_games
                     if not seq_solves_game(solve_game(np.array(t)), np.array(t))),
                    (t for t in unsolvable_games
                     if solve_game(np.array(t)) is not None)), None)
    if test is not None:
        print('Четвёртое задание не прошло тест:')
        print(np.array(test))
    else:
        print('Четвёртое задание прошло все тесты')

In [6]:
test_all()

Первое задание прошло все тесты
Второе задание прошло все тесты
Третье задание прошло все тесты
Четвёртое задание прошло все тесты
